In [1]:
# Packages
import pandas as pd
import numpy as np

# Local package costau_byw (welsh for cost of living)
import utils as ut

In [2]:
# Importing data
df_expenditure =  pd.read_csv("../data/csv/hh_expenditure_2001_2021.csv")
df_inflation = pd.read_csv("../data/csv/cpi_index.csv")

In [3]:

# Correcting the column names
initial_col_names = df_expenditure.columns
corrected_col_names = [x[0:4] for x in initial_col_names]
corrected_col_names[0] = 'category'
df_expenditure.columns = corrected_col_names

# Tidying the format
df_expenditure_formatted= pd.DataFrame(pd.melt(df_expenditure,
                       ["category"],
                       var_name="year",
                       value_name="expenditure"))

df_expenditure_formatted['year'] = df_expenditure_formatted['year'].astype('int64')
df_merged = pd.merge(df_expenditure_formatted,df_inflation, how="left", on = 'year')
df_merged= ut.adjust_df_for_inflation(df_merged, "expenditure", 'adjusted_expenditure', 'cpi_index_2015')

df_clean = df_merged.drop(['expenditure', 'cpi_index_2015'], axis = 1)
# Saving
df_clean.to_csv("../data/csv/df.csv", index=False)

#Caculating percentages for each category and year
df = df_clean[df_clean['category'] != 'Average per person']
years = df['year'].unique()
categories = df['category'].unique()

list_all = []
for year in years:
    percentage_list = []
    subsetted_year = df[df['year'] == year]
    total_exp = sum(subsetted_year['adjusted_expenditure']) 
    for cat in categories:
        subsetted_cat = subsetted_year[subsetted_year['category'] == cat]
        percentage = round(sum(subsetted_cat['adjusted_expenditure']) / total_exp * 100,2)
        row = [cat, percentage, year]
        list_all.append(row)
        
df_percentage = pd.DataFrame(list_all, columns= ['category', 'percentage', 'year'])
# Saving
df_percentage.to_csv("../data/csv/df_percentages.csv", index=False)
